In [1]:
%load_ext autoreload

# --------------- #
# region: Imports #
import os
import sys
module_path = os.path.abspath('../../..')
if module_path not in sys.path:
  sys.path.insert(0, module_path)
# endregion       #
# --------------- #

### Human player test for the model

In [2]:
import numpy as np
from matplotlib import pyplot as plt
from IPython.display import clear_output

# leakyemotion project imports
from examples.leakyemotions.agents import LeakyEmotionAgent, Wolf
from examples.leakyemotions.env import LeakyemotionsEnv
from examples.leakyemotions.custom_observation_spec import OneHotObservationSpec
from examples.leakyemotions.entities import Bush
from examples.leakyemotions.wolf_model import WolfModel

# sorrel imports
from sorrel.config import load_config, argparse
from sorrel.models.human_player import HumanPlayer
from sorrel.action.action_spec import ActionSpec
from sorrel.utils.visualization import (animate, image_from_array,
                                        visual_field_sprite)

cfg = load_config(argparse.Namespace(config='../configs/config.yaml'))

N_AGENTS = 2
MAX_TURNS = 100
EPSILON_DECAY = 0.0001
ENTITY_LIST = ["EmptyEntity", "Wall", "Grass", "Bush", "LeakyEmotionAgent", "Wolf"]

agents: list[LeakyEmotionAgent] = []
models: list[HumanPlayer] = []

world_height = 10
world_width = 10
spawn_prob = 0.002
agent_vision_radius = 2

observation_spec = OneHotObservationSpec(
            ENTITY_LIST, vision_radius=agent_vision_radius
        )
observation_spec.override_input_size(
    np.array(observation_spec.input_size).reshape(1, -1)
)
action_spec = ActionSpec(["up", "down", "left", "right"])


for i in range(N_AGENTS):
  models.append(
    HumanPlayer(
      input_size=[3, cfg.env.height*16, cfg.env.width*16, 4],
      action_space=6,
      memory_size=1,
      show=False
    )
  )

  agents.append(
      LeakyEmotionAgent(
          observation_spec=observation_spec, action_spec=action_spec, model=models[i], location=None
      )
  )
  agents.append(
      Wolf(
          observation_spec=observation_spec, action_spec=action_spec, model=WolfModel(1, 4, 1), location=None
      )
  )
  
env = LeakyemotionsEnv(
        world_height, world_width, spawn_prob, MAX_TURNS, agents)
env.reset()


In [3]:
print(env.world.shape)

(10, 10, 3)


### Turn taking loop
Choose an action from [0, 1, 2, 3, 4, 5] to act on the environment.

In [4]:
while True and not env.game_ended:
  img = visual_field_sprite(env)
  for i in range(len(img)):
    plt.imshow(img[i])
    plt.show()
  clear_output(wait=True)

  for agent in agents:
    if agent.kind == "Wolf":
      agent.asleep = True
  
  env.take_turn()
  

Please try again. Possible actions are below.
[0 1 2 3 4 5]


UnboundLocalError: cannot access local variable 'action' where it is not associated with a value